##### BJARKE BRODIN - INDBS 2020

# Indexing

Indexes can be used to tweak performance of sql queries.

To experiment with this we can run some queries with and without indexes and check the execution time.

In [28]:
drop INDEX if exists year;

Commands completed successfully

Total execution time: 00:00:00.006

In [29]:
create index year on movie(year);

Commands completed successfully

Total execution time: 00:00:00.026

In [30]:
select count(*)
from movie
where year=1948;

(1 row(s) affected)

Total execution time: 00:00:00.005

count


In [31]:
select count(*)
from movie
where year=1920 or year=1924 or year=1928 or year=1932 or year=1936 or year=1940 or year=1944 or year=1948;

(1 row(s) affected)

Total execution time: 00:00:00.020

count


## Improving performance with indexes

By indexing columns we can improve caching of much used data as well as providing fast small range and point lookup times.

Historically DBMS performance has suffered from having to read in random order, and having to read the same page often. Even on an SSD we may have to do more IO on random reads because we can only fetch a page at a time from disk.

If we have no indexes, we need to do <strong>full table scans</strong> to filter our data, we say that a full table scan makes sense if we return 80% of the relation.

When we do <strong>point queries</strong> or selective <strong>range queries</strong> however, having to do a full table scan is orders of growth slower, since we could easily do this lightning fast using search algorithms.

We can speed up range queries by way of an index-scan, we can speed up point queries by way of index-lookups.

We can implement indexes in a disk-page optimized way using B+ trees.

Primary / secondary indices - indices for primary keys and any column constrained by uniqueness

Clustered / unclustered indices - data order is identical to index order / data order is not necessarily order of index, this can affect performance greatly because of the effect on random/sequential IO.

## CLUSTERING IS VERY IMPORTANT

Unclustered indices are fine for point queries, but generally clustered implies more sequential IO which means less IOs (because prefetching will grab a page at a time and you will avoid having to read the same page more than once due to unlucky prefetches).<span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif;">Clustered indices are constrained by the fact, that clustering the data by some attribute prevents you from clustering it on a different attribute - the new sort would overwrite the old sort...</span>

### Retrieve M records, where M small:

Clustered - Probably 1 read

Unclustered - Probably M random reads

### Retrieve M recores, where M large:

Clustered - Probably M/pages sequental reads

Unclustered - probably M random reads

## Index scan vs full table scan

full table scan is constant - always O(n)

Index scans on clustered indices are typically dominated by the amount of items returned

Index scans on unclustered indices are fine for very narrow queries, but become very slow very fast if the return range grows.

## Covering indices

We say that an index covering all attributes used in a query is called <em>covering</em>. Covering indices have the effect that queries can be done index-only, since the index values must be in ram, we never need to IO, eliminating the penalty of using unclustered indices. This means that covering indices should be unclustered because the clustering simply doesnt improve performance on a covering index.